In [ ]:
import pandas as pd
import json
import pickle
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from uuid import uuid4

def split_docs(documents, filepath, chunk_size=400, chunk_overlap=40, seperators=['\n\n\n', '\n\n'], force_split=False):
    if os.path.exists(filepath) and not force_split:
        return pickle.load(open(filepath, 'rb'))

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=seperators
    )
    split_docs = splitter.split_documents(documents)
    for chunk in split_docs:
        chunk.metadata['uuid'] = str(uuid4())

    pickle.dump(split_docs, open(filepath, 'wb'))

    return split_docs
documents = PyPDFLoader("data/2024全球经济金融展望报告.pdf").load()
splitted_docs = split_docs(documents, os.path.join("outputs", 'split_docs.pkl'))

In [2]:
qa_df = pd.read_json(os.path.join("outputs", 'qa.json'), lines=True)
qa_df.head()

,question,context,answer,uuid
0,2023年全球经济增长的主要特点是什么？,2023 年全球经济增长动力持续回落，各国复苏分化，发达经济体增速明显放缓，新兴经济体整体表...,2023年全球经济增长动力持续回落，发达经济体增速明显放缓，而新兴经济体整体表现稳定。,35e5de15-d070-498f-8ffc-447971614d05
1,2024年全球经济复苏的预期是什么？,展望 2024 年，预计全球经济复苏将依旧疲软，主要经济体增长态势和货币政策走势将进一步分化。,预计全球经济复苏将依旧疲软，主要经济体的增长态势和货币政策走势将进一步分化。,35e5de15-d070-498f-8ffc-447971614d05
2,2023年全球股市的表现如何？,全球股市表现总体好于预期，但区域分化明显。,全球股市表现总体好于预期，但存在明显的区域分化。,35e5de15-d070-498f-8ffc-447971614d05
3,高利率环境对债券市场有什么影响？,高利率环境抑制债券融资需求，债券违约风险持续上升。,高利率环境抑制债券融资需求，并导致债券违约风险持续上升。,35e5de15-d070-498f-8ffc-447971614d05
4,预计2024年美国房地产市场将受到什么影响？,高利率和高债务对美国房地产市场脆弱性的影响等热点问题值得关注。,高利率和高债务将对美国房地产市场的脆弱性产生影响。,35e5de15-d070-498f-8ffc-447971614d05


In [6]:
def build_qa_samples(df, neg_batch_size=-1, n_neg_batch=5):
    from tqdm.auto import tqdm
    import math
    result = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        question = row['question']
        answer = row['context']
        neg_samples = df[df['question'] != question]['context'].values.tolist()
        n_neg_batch = min(n_neg_batch, math.ceil((len(df) - 1) / neg_batch_size))
        for neg_batch_idx in range(n_neg_batch):
            batch_neg_samples = neg_samples[neg_batch_idx * neg_batch_size : (neg_batch_idx + 1) * neg_batch_size]
            batch_neg_samples = [a  for a in batch_neg_samples if a != answer]
            result.append(
                {
                    'query': question,
                    'pos': [answer],
                    'neg': batch_neg_samples
                }
            )
    return result

neg_pos_samples = build_qa_samples(qa_df, neg_batch_size=16, n_neg_batch=32)
print(neg_pos_samples[0])


100%|██████████| 401/401 [00:00<00:00, 2674.45it/s]

{'query': '2023年全球经济增长的主要特点是什么？', 'pos': ['2023 年全球经济增长动力持续回落，各国复苏分化，发达经济体增速明显放缓，新兴经济体整体表现稳定。全球贸易增长乏力，各国生产景气度逐渐回落，内需对经济的拉动作用减弱。'], 'neg': ['展望 2024 年，预计全球经济复苏将依旧疲软，主要经济体增长态势和货币政策走势将进一步分化。', '全球股市表现总体好于预期，但区域分化明显。', '高利率环境抑制债券融资需求，债券违约风险持续上升。', '高利率和高债务对美国房地产市场脆弱性的影响等热点问题值得关注。', '国际原油市场短缺格局或延续，新能源发展成为重点。', '欧美央行大概率结束本轮紧缩货币周期，美元指数将逐步走弱。', '流向新兴经济体的跨境资本将增加。', '2023 年，全球经济增长动力持续回落。分区域看，各国复苏存在较大差异，发达经济体增速明显放缓，新兴经济体增速与2022 年大致持平。', '预计2023年全球GDP 增速为2.7%（市场汇率法），较2022 年下降0.3 个百分点。', '生产端，全球供应链持续恢复，但生产景气度逐渐回落。', '欧美央行货币政策延续收紧态势，但步伐整体放缓。', '高利率环境抑制债券融资需求，债券违约风险持续上升。', '展望2024 年，预计全球经济复苏将依旧疲软，主要经济体增长态势和货币政策将进一步分化。', '日本央行可能退出负利率政策，跨境资本回流美国趋势将放缓。', '本期报告分别对海湾六国经济发展与投资前景、高利率和高债务对美国房地产市场脆弱性的影响两个专题展开分析。', '分区域看，全球经济复苏不均衡，各国存在较大差异。发达经济体增速明显放缓，预计2023年增速较2022年下降1个百分点。']}


In [7]:
write_file = os.path.join("outputs", 'neg_pos.json')
with open(write_file, 'wt') as file:
    for dict in neg_pos_samples:
        file.write(json.dumps(dict, ensure_ascii=False) + '\n')